In [1]:
!pip install --upgrade git+https://github.com/allenai/ir_datasets.git@crisisfacts # install ir_datasets (crisisfacts branch)


ERROR: Invalid requirement: '#'


In [2]:
import pandas as pd
import re
import nltk
import json


In [3]:

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Kumar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kumar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Kumar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
def preprocess_text(text):
    # Step 1: Convert to lowercase
    text = text.lower()

    # Step 2: Tokenization
    tokens = nltk.word_tokenize(text)

    # Step 3: Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]

    # Step 4: Remove special characters and punctuation
    filtered_tokens = [re.sub(r'[^a-zA-Z0-9]', '', word) for word in filtered_tokens]

    # Step 5: Lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]

    # Step 6: Join the tokens back into a cleaned text
    cleaned_text = ' '.join(lemmatized_tokens)

    return cleaned_text

In [5]:
# credentials = {
#     "institution": "<University/Agency Name>", # University, Company or Public Agency Name
#     "contactname": "<Your Name>", # Your Name
#     "email": "<Your Email>", # A contact email address
#     "institutiontype": "<Research | Industry | Public Sector>" # Either 'Research', 'Industry', or 'Public Sector'
# }

# # Write this to a file so it can be read when needed
# import json
# import os

# home_dir = os.path.expanduser('~')

# !mkdir -p ~/.ir_datasets/auth/
# with open(home_dir + '/.ir_datasets/auth/crisisfacts.json', 'w') as f:
#     json.dump(credentials, f)

In [6]:
# Event numbers as a list
eventNoList = [
        "001", # Lilac Wildfire 2017
        "002", # Cranston Wildfire 2018
        "003", # Holy Wildfire 2018
        "004", # Hurricane Florence 2018
        "005", # 2018 Maryland Flood
        "006", # Saddleridge Wildfire 2019
        "007", # Hurricane Laura 2020
        "008", # Hurricane Sally 2020
        "009", # Beirut Explosion, 2020
        "010", # Houston Explosion, 2020
        "011", # Rutherford TN Floods, 2020
        "012", # TN Derecho, 2020
        "013", # Edenville Dam Fail, 2020
        "014", # Hurricane Dorian, 2019
        "015", # Kincade Wildfire, 2019
        "016", # Easter Tornado Outbreak, 2020
        "017", # Tornado Outbreak, 2020 Apr
        "018", # Tornado Outbreak, 2020 March
]

In [7]:

import requests

# Gets the list of days for a specified event number, e.g. '001'
def getDaysForEventNo(eventNo):

    # We will download a file containing the day list for an event
    url = "http://trecis.org/CrisisFACTs/CrisisFACTS-"+eventNo+".requests.json"

    # Download the list and parse as JSON
    dayList = requests.get(url).json()
    # Print each day
    # Note each day object contains the following fields
    #   {
    #      "eventID" : "CrisisFACTS-001",
    #      "requestID" : "CrisisFACTS-001-r3",
    #      "dateString" : "2017-12-07",
    #      "startUnixTimestamp" : 1512604800,
    #      "endUnixTimestamp" : 1512691199
    #   }

    return dayList

for day in getDaysForEventNo(eventNoList[0]):
    print(day["dateString"])

2017-12-07
2017-12-08
2017-12-09
2017-12-10
2017-12-11
2017-12-12
2017-12-13
2017-12-14
2017-12-15


In [8]:
!pip install --upgrade python-terrier # install pyTerrier

ERROR: Invalid requirement: '#'


# Simple query-based baseline


In [9]:
eventsMeta = {}

for eventNo in eventNoList: # for each event
    print("Event "+eventNo)
    
    dailyInfo = getDaysForEventNo(eventNo) # get the list of days
    eventsMeta[eventNo]= dailyInfo
    for day in dailyInfo: # for each day
        print("  crisisfacts/"+eventNo+"/"+day["dateString"], "-->", day["requestID"]) # construct the request string

    print()

Event 001
  crisisfacts/001/2017-12-07 --> CrisisFACTS-001-r3
  crisisfacts/001/2017-12-08 --> CrisisFACTS-001-r4
  crisisfacts/001/2017-12-09 --> CrisisFACTS-001-r5
  crisisfacts/001/2017-12-10 --> CrisisFACTS-001-r6
  crisisfacts/001/2017-12-11 --> CrisisFACTS-001-r7
  crisisfacts/001/2017-12-12 --> CrisisFACTS-001-r8
  crisisfacts/001/2017-12-13 --> CrisisFACTS-001-r9
  crisisfacts/001/2017-12-14 --> CrisisFACTS-001-r10
  crisisfacts/001/2017-12-15 --> CrisisFACTS-001-r11

Event 002
  crisisfacts/002/2018-07-25 --> CrisisFACTS-002-r1
  crisisfacts/002/2018-07-26 --> CrisisFACTS-002-r2
  crisisfacts/002/2018-07-27 --> CrisisFACTS-002-r3
  crisisfacts/002/2018-07-28 --> CrisisFACTS-002-r4
  crisisfacts/002/2018-07-29 --> CrisisFACTS-002-r5
  crisisfacts/002/2018-07-30 --> CrisisFACTS-002-r6

Event 003
  crisisfacts/003/2018-08-06 --> CrisisFACTS-003-r5
  crisisfacts/003/2018-08-07 --> CrisisFACTS-003-r6
  crisisfacts/003/2018-08-08 --> CrisisFACTS-003-r7
  crisisfacts/003/2018-08-09 -

In [10]:
import pandas as pd
import numpy as np
import pyterrier as pt
import json
import ir_datasets

In [11]:
def get_retriever(datasetID):
    # Initalize pyTerrier if not started
    if not pt.started():
        pt.init()

    # Ask pyTerrier to download the dataset, the 'irds:' header tells pyTerrier to use ir_datasets as the data source
    pyTerrierDataset = pt.get_dataset(f'irds:{datasetID}')
    # To create the index, we use an 'indexer', this interates over the documents in the collection and adds them to the index
    # The paramters of this call are:
    #  Index Storage Path: "None" (some index types write to disk, this would be the directory to write to)
    #  Index Type: type=pt.index.IndexingType(3) (Type 3 is a Memory Index)
    #  Meta Index Fields: meta=['docno', 'text'] (The index also can store raw fields so they can be attached to the search results, this specifies what fields to store)
    #  Meta Index Lengths: meta_lengths=[40, 200] (pyTerrier allocates a fixed amount of storage space per field, how many characters should this be?)
    indexer = pt.IterDictIndexer(
        "None", 
        type=pt.index.IndexingType(3), 
        meta=['docno', 'text'], 
        meta_lengths=[40, 200],
    )

    # Trigger the indexing process
    index = indexer.index(pyTerrierDataset.get_corpus_iter())
    retriever = pt.BatchRetrieve(
        index, 
#         wmodel="DFReeKLIM",
#         wmodel="TF_IDF",
        wmodel="BM25",
#         wmodel="PL2",
        metadata=["docno", "text"],
    )
    return retriever

def  get_quries_hits(retriever, queries):
    
    event_queries={}
    event_scores={}
    
    for index, query in queries.iterrows():
#         results = pd.DataFrame(retriever.search(query['indicative_terms']))
        results = pd.DataFrame(retriever.search(preprocess_text(query['text']+" "+query['indicative_terms'])))
#         results = pd.DataFrame(retriever.search(query['text']+" "+query['indicative_terms']))
        for ind, result in results.iterrows():
            if result['docno'] in event_queries:
                event_queries[result['docno']].append(query['query_id'])
                event_scores[result['docno']].append(result['score'])
            else:
                event_queries[result['docno']]=[query['query_id']]
                event_scores[result['docno']]=[result['score']]
    
    return event_queries,event_scores

In [12]:
# Build the document cache for each CrisisFACTS event-day
#. request. All data goes to the `ir_datasets/crisisfacts` directory,
#. which on Unix-type machines is `~/.ir_datasets/crisisfacts/`
for eventId,dailyInfo in eventsMeta.items():

    for thisDay in dailyInfo:
        
        requestID = thisDay["requestID"]
        print("\tRequest-ID: ",requestID)
        ir_dataset_id = "crisisfacts/%s/%s" % (eventId, thisDay["dateString"])        
        
        dataset = ir_datasets.load(ir_dataset_id)
        print(dataset.dataset_id())
        
        itemsAsDataFrame = pd.DataFrame(dataset.docs_iter())
        print("\tDataset Size:", itemsAsDataFrame.shape[0])
        
        # This should be true. If you get an error here, 
        #. it suggests a bug popped up during data download. 
        #. Try re-running this cell.
        assert itemsAsDataFrame.shape[0] > 0
        
        queries = pd.DataFrame(dataset.queries_iter())
        print("\tQuery Count:", queries.shape[0])
        
        # This should also be true. If you get an error here, 
        #. it suggests a bug popped up during data download. 
        #. Try re-running this cell.
        assert queries.shape[0] > 0
        
        # If the above code fails on JSON decode error associated with queries,
        #. that means you've ended up with an empty query file. We have an 
        #. intermittent bug in the networking infrastructure that occasionally
        #. causes this. You can look for 0-byte files in ~/.ir_datasets/crisisfacts
        #. and delete them to force the library to retry downloading
        #. E.g., `rm ~/.ir_datasets/crisisfacts/2020-08-05/009/queries.json`

	Request-ID:  CrisisFACTS-001-r3
crisisfacts/001/2017-12-07
	Dataset Size: 7288
	Query Count: 52
	Request-ID:  CrisisFACTS-001-r4
crisisfacts/001/2017-12-08
	Dataset Size: 19231
	Query Count: 52
	Request-ID:  CrisisFACTS-001-r5
crisisfacts/001/2017-12-09
	Dataset Size: 5839
	Query Count: 52
	Request-ID:  CrisisFACTS-001-r6
crisisfacts/001/2017-12-10
	Dataset Size: 4407
	Query Count: 52
	Request-ID:  CrisisFACTS-001-r7
crisisfacts/001/2017-12-11
	Dataset Size: 3394
	Query Count: 52
	Request-ID:  CrisisFACTS-001-r8
crisisfacts/001/2017-12-12
	Dataset Size: 2805
	Query Count: 52
	Request-ID:  CrisisFACTS-001-r9
crisisfacts/001/2017-12-13
	Dataset Size: 2658
	Query Count: 52
	Request-ID:  CrisisFACTS-001-r10
crisisfacts/001/2017-12-14
	Dataset Size: 2728
	Query Count: 52
	Request-ID:  CrisisFACTS-001-r11
crisisfacts/001/2017-12-15
	Dataset Size: 2665
	Query Count: 52
	Request-ID:  CrisisFACTS-002-r1
crisisfacts/002/2018-07-25
	Dataset Size: 5056
	Query Count: 52
	Request-ID:  CrisisFACTS-0

	Dataset Size: 2928
	Query Count: 48
	Request-ID:  CrisisFACTS-013-r5
crisisfacts/013/2020-05-28
	Dataset Size: 315
	Query Count: 48
	Request-ID:  CrisisFACTS-013-r6
crisisfacts/013/2020-05-29
	Dataset Size: 13
	Query Count: 48
	Request-ID:  CrisisFACTS-014-r0
crisisfacts/014/2019-08-29
	Dataset Size: 54000
	Query Count: 51
	Request-ID:  CrisisFACTS-014-r1
crisisfacts/014/2019-08-30
	Dataset Size: 51000
	Query Count: 51
	Request-ID:  CrisisFACTS-014-r2
crisisfacts/014/2019-08-31
	Dataset Size: 7000
	Query Count: 51
	Request-ID:  CrisisFACTS-014-r3
crisisfacts/014/2019-09-01
	Dataset Size: 105098
	Query Count: 51
	Request-ID:  CrisisFACTS-014-r4
crisisfacts/014/2019-09-02
	Dataset Size: 113397
	Query Count: 51
	Request-ID:  CrisisFACTS-014-r5
crisisfacts/014/2019-09-03
	Dataset Size: 24056
	Query Count: 51
	Request-ID:  CrisisFACTS-014-r6
crisisfacts/014/2019-09-04
	Dataset Size: 76004
	Query Count: 51
	Request-ID:  CrisisFACTS-015-r0
crisisfacts/015/2019-10-25
	Dataset Size: 16379
	Que

In [13]:
# datasat = pd.read_csv('dtf_train.csv')
# query = pd.read_csv('query11.csv')

In [14]:
# dataset = ir_datasets.load(ir_dataset_id)
# dataset

In [15]:
for eventId,dailyInfo in eventsMeta.items():

    for thisDay in dailyInfo:
        
        requestID = thisDay["requestID"]
        ir_dataset_id = "crisisfacts/%s/%s" % (eventId, thisDay["dateString"])        
        
        # Should use the cached data we downloaded above
        dataset = ir_datasets.load(ir_dataset_id)        
        queries = pd.DataFrame(dataset.queries_iter())
        print(queries)

                     query_id  \
0    CrisisFACTS-General-q001   
1    CrisisFACTS-General-q002   
2    CrisisFACTS-General-q003   
3    CrisisFACTS-General-q004   
4    CrisisFACTS-General-q005   
5    CrisisFACTS-General-q006   
6    CrisisFACTS-General-q007   
7    CrisisFACTS-General-q008   
8    CrisisFACTS-General-q009   
9    CrisisFACTS-General-q010   
10   CrisisFACTS-General-q011   
11   CrisisFACTS-General-q012   
12   CrisisFACTS-General-q013   
13   CrisisFACTS-General-q014   
14   CrisisFACTS-General-q015   
15   CrisisFACTS-General-q016   
16   CrisisFACTS-General-q017   
17   CrisisFACTS-General-q018   
18   CrisisFACTS-General-q019   
19   CrisisFACTS-General-q020   
20   CrisisFACTS-General-q021   
21   CrisisFACTS-General-q022   
22   CrisisFACTS-General-q023   
23   CrisisFACTS-General-q024   
24   CrisisFACTS-General-q025   
25   CrisisFACTS-General-q026   
26   CrisisFACTS-General-q027   
27   CrisisFACTS-General-q028   
28   CrisisFACTS-General-q029   
29   Crisi

                     query_id  \
0    CrisisFACTS-General-q001   
1    CrisisFACTS-General-q002   
2    CrisisFACTS-General-q003   
3    CrisisFACTS-General-q004   
4    CrisisFACTS-General-q005   
5    CrisisFACTS-General-q006   
6    CrisisFACTS-General-q007   
7    CrisisFACTS-General-q008   
8    CrisisFACTS-General-q009   
9    CrisisFACTS-General-q010   
10   CrisisFACTS-General-q011   
11   CrisisFACTS-General-q012   
12   CrisisFACTS-General-q013   
13   CrisisFACTS-General-q014   
14   CrisisFACTS-General-q015   
15   CrisisFACTS-General-q016   
16   CrisisFACTS-General-q017   
17   CrisisFACTS-General-q018   
18   CrisisFACTS-General-q019   
19   CrisisFACTS-General-q020   
20   CrisisFACTS-General-q021   
21   CrisisFACTS-General-q022   
22   CrisisFACTS-General-q023   
23   CrisisFACTS-General-q024   
24   CrisisFACTS-General-q025   
25   CrisisFACTS-General-q026   
26   CrisisFACTS-General-q027   
27   CrisisFACTS-General-q028   
28   CrisisFACTS-General-q029   
29   Crisi

                     query_id  \
0    CrisisFACTS-General-q001   
1    CrisisFACTS-General-q002   
2    CrisisFACTS-General-q003   
3    CrisisFACTS-General-q004   
4    CrisisFACTS-General-q005   
5    CrisisFACTS-General-q006   
6    CrisisFACTS-General-q007   
7    CrisisFACTS-General-q008   
8    CrisisFACTS-General-q009   
9    CrisisFACTS-General-q010   
10   CrisisFACTS-General-q011   
11   CrisisFACTS-General-q012   
12   CrisisFACTS-General-q013   
13   CrisisFACTS-General-q014   
14   CrisisFACTS-General-q015   
15   CrisisFACTS-General-q016   
16   CrisisFACTS-General-q017   
17   CrisisFACTS-General-q018   
18   CrisisFACTS-General-q019   
19   CrisisFACTS-General-q020   
20   CrisisFACTS-General-q021   
21   CrisisFACTS-General-q022   
22   CrisisFACTS-General-q023   
23   CrisisFACTS-General-q024   
24   CrisisFACTS-General-q025   
25   CrisisFACTS-General-q026   
26   CrisisFACTS-General-q027   
27   CrisisFACTS-General-q028   
28   CrisisFACTS-General-q029   
29   Crisi

                      query_id  \
0     CrisisFACTS-General-q001   
1     CrisisFACTS-General-q002   
2     CrisisFACTS-General-q003   
3     CrisisFACTS-General-q004   
4     CrisisFACTS-General-q005   
5     CrisisFACTS-General-q006   
6     CrisisFACTS-General-q007   
7     CrisisFACTS-General-q008   
8     CrisisFACTS-General-q009   
9     CrisisFACTS-General-q010   
10    CrisisFACTS-General-q011   
11    CrisisFACTS-General-q012   
12    CrisisFACTS-General-q013   
13    CrisisFACTS-General-q014   
14    CrisisFACTS-General-q015   
15    CrisisFACTS-General-q016   
16    CrisisFACTS-General-q017   
17    CrisisFACTS-General-q018   
18    CrisisFACTS-General-q019   
19    CrisisFACTS-General-q020   
20    CrisisFACTS-General-q021   
21    CrisisFACTS-General-q022   
22    CrisisFACTS-General-q023   
23    CrisisFACTS-General-q024   
24    CrisisFACTS-General-q025   
25    CrisisFACTS-General-q026   
26    CrisisFACTS-General-q027   
27    CrisisFACTS-General-q028   
28    CrisisFA

                      query_id  \
0     CrisisFACTS-General-q001   
1     CrisisFACTS-General-q002   
2     CrisisFACTS-General-q003   
3     CrisisFACTS-General-q004   
4     CrisisFACTS-General-q005   
5     CrisisFACTS-General-q006   
6     CrisisFACTS-General-q007   
7     CrisisFACTS-General-q008   
8     CrisisFACTS-General-q009   
9     CrisisFACTS-General-q010   
10    CrisisFACTS-General-q011   
11    CrisisFACTS-General-q012   
12    CrisisFACTS-General-q013   
13    CrisisFACTS-General-q014   
14    CrisisFACTS-General-q015   
15    CrisisFACTS-General-q016   
16    CrisisFACTS-General-q017   
17    CrisisFACTS-General-q018   
18    CrisisFACTS-General-q019   
19    CrisisFACTS-General-q020   
20    CrisisFACTS-General-q021   
21    CrisisFACTS-General-q022   
22    CrisisFACTS-General-q023   
23    CrisisFACTS-General-q024   
24    CrisisFACTS-General-q025   
25    CrisisFACTS-General-q026   
26    CrisisFACTS-General-q027   
27    CrisisFACTS-General-q028   
28    CrisisFA

...

                    query_id  \
0   CrisisFACTS-General-q001   
1   CrisisFACTS-General-q002   
2   CrisisFACTS-General-q003   
3   CrisisFACTS-General-q004   
4   CrisisFACTS-General-q005   
5   CrisisFACTS-General-q006   
6   CrisisFACTS-General-q007   
7   CrisisFACTS-General-q008   
8   CrisisFACTS-General-q009   
9   CrisisFACTS-General-q010   
10  CrisisFACTS-General-q011   
11  CrisisFACTS-General-q012   
12  CrisisFACTS-General-q013   
13  CrisisFACTS-General-q014   
14  CrisisFACTS-General-q015   
15  CrisisFACTS-General-q016   
16  CrisisFACTS-General-q017   
17  CrisisFACTS-General-q018   
18  CrisisFACTS-General-q019   
19  CrisisFACTS-General-q020   
20  CrisisFACTS-General-q021   
21  CrisisFACTS-General-q022   
22  CrisisFACTS-General-q023   
23  CrisisFACTS-General-q024   
24  CrisisFACTS-General-q025   
25  CrisisFACTS-General-q026   
26  CrisisFACTS-General-q027   
27  CrisisFACTS-General-q028   
28  CrisisFACTS-General-q029   
29  CrisisFACTS-General-q030   
30  Cris

                      query_id  \
0     CrisisFACTS-General-q001   
1     CrisisFACTS-General-q002   
2     CrisisFACTS-General-q003   
3     CrisisFACTS-General-q004   
4     CrisisFACTS-General-q005   
5     CrisisFACTS-General-q006   
6     CrisisFACTS-General-q007   
7     CrisisFACTS-General-q008   
8     CrisisFACTS-General-q009   
9     CrisisFACTS-General-q010   
10    CrisisFACTS-General-q011   
11    CrisisFACTS-General-q012   
12    CrisisFACTS-General-q013   
13    CrisisFACTS-General-q014   
14    CrisisFACTS-General-q015   
15    CrisisFACTS-General-q016   
16    CrisisFACTS-General-q017   
17    CrisisFACTS-General-q018   
18    CrisisFACTS-General-q019   
19    CrisisFACTS-General-q020   
20    CrisisFACTS-General-q021   
21    CrisisFACTS-General-q022   
22    CrisisFACTS-General-q023   
23    CrisisFACTS-General-q024   
24    CrisisFACTS-General-q025   
25    CrisisFACTS-General-q026   
26    CrisisFACTS-General-q027   
27    CrisisFACTS-General-q028   
28    CrisisFA

                     query_id  \
0    CrisisFACTS-General-q001   
1    CrisisFACTS-General-q002   
2    CrisisFACTS-General-q003   
3    CrisisFACTS-General-q004   
4    CrisisFACTS-General-q005   
5    CrisisFACTS-General-q006   
6    CrisisFACTS-General-q007   
7    CrisisFACTS-General-q008   
8    CrisisFACTS-General-q009   
9    CrisisFACTS-General-q010   
10   CrisisFACTS-General-q011   
11   CrisisFACTS-General-q012   
12   CrisisFACTS-General-q013   
13   CrisisFACTS-General-q014   
14   CrisisFACTS-General-q015   
15   CrisisFACTS-General-q016   
16   CrisisFACTS-General-q017   
17   CrisisFACTS-General-q018   
18   CrisisFACTS-General-q019   
19   CrisisFACTS-General-q020   
20   CrisisFACTS-General-q021   
21   CrisisFACTS-General-q022   
22   CrisisFACTS-General-q023   
23   CrisisFACTS-General-q024   
24   CrisisFACTS-General-q025   
25   CrisisFACTS-General-q026   
26   CrisisFACTS-General-q027   
27   CrisisFACTS-General-q028   
28   CrisisFACTS-General-q029   
29   Crisi

                    query_id  \
0   CrisisFACTS-General-q001   
1   CrisisFACTS-General-q002   
2   CrisisFACTS-General-q003   
3   CrisisFACTS-General-q004   
4   CrisisFACTS-General-q005   
5   CrisisFACTS-General-q006   
6   CrisisFACTS-General-q007   
7   CrisisFACTS-General-q008   
8   CrisisFACTS-General-q009   
9   CrisisFACTS-General-q010   
10  CrisisFACTS-General-q011   
11  CrisisFACTS-General-q012   
12  CrisisFACTS-General-q013   
13  CrisisFACTS-General-q014   
14  CrisisFACTS-General-q015   
15  CrisisFACTS-General-q016   
16  CrisisFACTS-General-q017   
17  CrisisFACTS-General-q018   
18  CrisisFACTS-General-q019   
19  CrisisFACTS-General-q020   
20  CrisisFACTS-General-q021   
21  CrisisFACTS-General-q022   
22  CrisisFACTS-General-q023   
23  CrisisFACTS-General-q024   
24  CrisisFACTS-General-q025   
25  CrisisFACTS-General-q026   
26  CrisisFACTS-General-q027   
27  CrisisFACTS-General-q028   
28  CrisisFACTS-General-q029   
29  CrisisFACTS-General-q030   
30  Cris

                    query_id  \
0   CrisisFACTS-General-q001   
1   CrisisFACTS-General-q002   
2   CrisisFACTS-General-q003   
3   CrisisFACTS-General-q004   
4   CrisisFACTS-General-q005   
5   CrisisFACTS-General-q006   
6   CrisisFACTS-General-q007   
7   CrisisFACTS-General-q008   
8   CrisisFACTS-General-q009   
9   CrisisFACTS-General-q010   
10  CrisisFACTS-General-q011   
11  CrisisFACTS-General-q012   
12  CrisisFACTS-General-q013   
13  CrisisFACTS-General-q014   
14  CrisisFACTS-General-q015   
15  CrisisFACTS-General-q016   
16  CrisisFACTS-General-q017   
17  CrisisFACTS-General-q018   
18  CrisisFACTS-General-q019   
19  CrisisFACTS-General-q020   
20  CrisisFACTS-General-q021   
21  CrisisFACTS-General-q022   
22  CrisisFACTS-General-q023   
23  CrisisFACTS-General-q024   
24  CrisisFACTS-General-q025   
25  CrisisFACTS-General-q026   
26  CrisisFACTS-General-q027   
27  CrisisFACTS-General-q028   
28  CrisisFACTS-General-q029   
29  CrisisFACTS-General-q030   
30  Cris

In [16]:
print(queries.info())
print("\t")
queries

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   query_id                 51 non-null     object
 1   text                     51 non-null     object
 2   indicative_terms         51 non-null     object
 3   trecis_category_mapping  51 non-null     object
 4   event_id                 51 non-null     object
 5   event_title              51 non-null     object
 6   event_dataset            51 non-null     object
 7   event_description        51 non-null     object
 8   event_trecis_id          51 non-null     object
 9   event_type               51 non-null     object
 10  event_url                51 non-null     object
dtypes: object(11)
memory usage: 4.5+ KB
None
	


,query_id,text,indicative_terms,trecis_category_mapping,event_id,event_title,event_dataset,event_description,event_trecis_id,event_type,event_url
0,CrisisFACTS-General-q001,Have airports closed,airport closed,Report-Factoid,CrisisFACTS-018,2020 Tornado Outbreak of March,2020_tornado_outbreak_of_march,A small but deadly tornado outbreak affected W...,TRECIS-CTIT-H-120,Tornado,https://en.wikipedia.org/wiki/Tornado_outbreak...
1,CrisisFACTS-General-q002,Have railways closed,rail closed,Report-Factoid,CrisisFACTS-018,2020 Tornado Outbreak of March,2020_tornado_outbreak_of_march,A small but deadly tornado outbreak affected W...,TRECIS-CTIT-H-120,Tornado,https://en.wikipedia.org/wiki/Tornado_outbreak...
2,CrisisFACTS-General-q003,Have water supplied been contaminated,water supply,Report-EmergingThreats,CrisisFACTS-018,2020 Tornado Outbreak of March,2020_tornado_outbreak_of_march,A small but deadly tornado outbreak affected W...,TRECIS-CTIT-H-120,Tornado,https://en.wikipedia.org/wiki/Tornado_outbreak...
3,CrisisFACTS-General-q004,How many firefighters are active,firefighters on-duty,Report-Factoid,CrisisFACTS-018,2020 Tornado Outbreak of March,2020_tornado_outbreak_of_march,A small but deadly tornado outbreak affected W...,TRECIS-CTIT-H-120,Tornado,https://en.wikipedia.org/wiki/Tornado_outbreak...
4,CrisisFACTS-General-q005,How many people are affected,evacuated,Report-Factoid,CrisisFACTS-018,2020 Tornado Outbreak of March,2020_tornado_outbreak_of_march,A small but deadly tornado outbreak affected W...,TRECIS-CTIT-H-120,Tornado,https://en.wikipedia.org/wiki/Tornado_outbreak...
5,CrisisFACTS-General-q006,How many people are in shelters,shelters,Report-Factoid,CrisisFACTS-018,2020 Tornado Outbreak of March,2020_tornado_outbreak_of_march,A small but deadly tornado outbreak affected W...,TRECIS-CTIT-H-120,Tornado,https://en.wikipedia.org/wiki/Tornado_outbreak...
6,CrisisFACTS-General-q007,How many people are missing,missing,Report-Factoid,CrisisFACTS-018,2020 Tornado Outbreak of March,2020_tornado_outbreak_of_march,A small but deadly tornado outbreak affected W...,TRECIS-CTIT-H-120,Tornado,https://en.wikipedia.org/wiki/Tornado_outbreak...
7,CrisisFACTS-General-q008,How many people are trapped,trapped,Request-SearchAndRescue,CrisisFACTS-018,2020 Tornado Outbreak of March,2020_tornado_outbreak_of_march,A small but deadly tornado outbreak affected W...,TRECIS-CTIT-H-120,Tornado,https://en.wikipedia.org/wiki/Tornado_outbreak...
8,CrisisFACTS-General-q009,How many people have been injured,injury injured,Report-Factoid,CrisisFACTS-018,2020 Tornado Outbreak of March,2020_tornado_outbreak_of_march,A small but deadly tornado outbreak affected W...,TRECIS-CTIT-H-120,Tornado,https://en.wikipedia.org/wiki/Tornado_outbreak...
9,CrisisFACTS-General-q010,How many people have been killed,killed dead,Report-Factoid,CrisisFACTS-018,2020 Tornado Outbreak of March,2020_tornado_outbreak_of_march,A small but deadly tornado outbreak affected W...,TRECIS-CTIT-H-120,Tornado,https://en.wikipedia.org/wiki/Tornado_outbreak...


In [18]:
rows = []

for eventId,dailyInfo in eventsMeta.items():

    for thisDay in dailyInfo:
        
        requestID = thisDay["requestID"]
        ir_dataset_id = "crisisfacts/%s/%s" % (eventId, thisDay["dateString"])        
        
        # Should use the cached data we downloaded above
        dataset = ir_datasets.load(ir_dataset_id)        
        queries = pd.DataFrame(dataset.queries_iter())
#         queries = query

        retriever = get_retriever(ir_dataset_id)
        event_queries, event_scores = get_quries_hits(retriever,queries)
        
        itemsAsDataFrame = pd.DataFrame(dataset.docs_iter())
        for key,value in event_scores.items():
            fact = itemsAsDataFrame.query(f"doc_id== '{key}'").iloc[0]
            
            rows.append({
                'requestID' : requestID, 
                'factText' : fact["text"], 
                'unixTimestamp' : fact["unix_timestamp"],
                "importance_count" : len(value),
                "importance_sum" : np.sum(value),
                "sources" : [key],
                "streamID" : key,
                "informationNeeds" : event_queries[key]
            })
        print("successull for " + requestID)
        dtf = pd.DataFrame(rows)
        print(dtf)
        break
    break

output = pd.DataFrame(rows)


crisisfacts/001/2017-12-07 documents: 0it [00:00, ?it/s]
crisisfacts/001/2017-12-07 documents: 306it [00:00, 2842.50it/s]
crisisfacts/001/2017-12-07 documents: 591it [00:00, 2260.03it/s]
crisisfacts/001/2017-12-07 documents: 824it [00:00, 2128.05it/s]
crisisfacts/001/2017-12-07 documents: 1040it [00:00, 2023.58it/s]
crisisfacts/001/2017-12-07 documents: 1244it [00:00, 1829.79it/s]
crisisfacts/001/2017-12-07 documents: 1429it [00:00, 1641.82it/s]
crisisfacts/001/2017-12-07 documents: 1596it [00:00, 1466.48it/s]
crisisfacts/001/2017-12-07 documents: 1746it [00:01, 1420.50it/s]
crisisfacts/001/2017-12-07 documents: 1910it [00:01, 1428.07it/s]
crisisfacts/001/2017-12-07 documents: 2055it [00:01, 1430.06it/s]
crisisfacts/001/2017-12-07 documents: 2291it [00:01, 1615.50it/s]
crisisfacts/001/2017-12-07 documents: 2454it [00:01, 1613.33it/s]
crisisfacts/001/2017-12-07 documents: 2661it [00:01, 1735.63it/s]
crisisfacts/001/2017-12-07 documents: 2877it [00:01, 1783.00it/s]
crisisfacts/001/2017-

17:42:13.103 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.


successull for CrisisFACTS-001-r3
               requestID                                           factText  \
0     CrisisFACTS-001-r3  Good to be home again! @ San Diego Internation...   
1     CrisisFACTS-001-r3  #LilacFire Smokey here in Oceanside above the ...   
2     CrisisFACTS-001-r3  #lilacFire photos from Palomar airport rd Carl...   
3     CrisisFACTS-001-r3  Back to Cali🌈 . . . #california #sandiego #カリフ...   
4     CrisisFACTS-001-r3  I'm at San Diego International Airport in San ...   
...                  ...                                                ...   
4016  CrisisFACTS-001-r3  @mollysierra_ I signed up for the Nixle Alerts...   
4017  CrisisFACTS-001-r3  Mr. Holt has spent the last four days shuttlin...   
4018  CrisisFACTS-001-r3  We have a few fires, some heading our way, wav...   
4019  CrisisFACTS-001-r3  Scooter clips ft new song!!!!!!!!!F the world ...   
4020  CrisisFACTS-001-r3  @gretchenbostrom Thank you. Safe and sound her...   

      unixTimesta

In [32]:
rows
output = pd.DataFrame(rows)

In [33]:
output.to_csv('Output_BM25_Emb_WP.csv')

In [34]:
output

,requestID,factText,unixTimestamp,importance_count,importance_sum,sources,streamID,informationNeeds
0,CrisisFACTS-001-r3,Good to be home again! @ San Diego Internation...,1512632372,2,16.634734,[CrisisFACTS-001-Twitter-48717-0],CrisisFACTS-001-Twitter-48717-0,"[CrisisFACTS-General-q001, CrisisFACTS-Wildfir..."
1,CrisisFACTS-001-r3,#LilacFire Smokey here in Oceanside above the ...,1512681837,2,17.147703,[CrisisFACTS-001-Twitter-5831-0],CrisisFACTS-001-Twitter-5831-0,"[CrisisFACTS-General-q001, CrisisFACTS-Wildfir..."
2,CrisisFACTS-001-r3,#lilacFire photos from Palomar airport rd Carl...,1512679976,1,9.929671,[CrisisFACTS-001-Twitter-512-0],CrisisFACTS-001-Twitter-512-0,[CrisisFACTS-General-q001]
3,CrisisFACTS-001-r3,Back to Cali🌈 . . . #california #sandiego #カリフ...,1512609067,1,9.529182,[CrisisFACTS-001-Twitter-22676-0],CrisisFACTS-001-Twitter-22676-0,[CrisisFACTS-General-q001]
4,CrisisFACTS-001-r3,I'm at San Diego International Airport in San ...,1512605323,1,9.159746,[CrisisFACTS-001-Twitter-49822-0],CrisisFACTS-001-Twitter-49822-0,[CrisisFACTS-General-q001]
...,...,...,...,...,...,...,...,...
466774,CrisisFACTS-018-r5,"If you think differently, go propose a new tax...",1583540053,1,4.458054,[CrisisFACTS-018-Reddit-7515-2],CrisisFACTS-018-Reddit-7515-2,[CrisisFACTS-General-q040]
466775,CrisisFACTS-018-r5,You could get better at running with conscious...,1583585050,1,2.679004,[CrisisFACTS-018-Reddit-7533-2],CrisisFACTS-018-Reddit-7533-2,[CrisisFACTS-General-q041]
466776,CrisisFACTS-018-r5,"They turned me away today, said they had too m...",1583544469,1,6.346459,[CrisisFACTS-018-Reddit-7521-0],CrisisFACTS-018-Reddit-7521-0,[CrisisFACTS-General-q044]
466777,CrisisFACTS-018-r5,Tornados are over and done quickly.,1583562502,4,26.491371,[CrisisFACTS-018-Reddit-7528-6],CrisisFACTS-018-Reddit-7528-6,"[CrisisFACTS-Tornado-q002, CrisisFACTS-Tornado..."


In [35]:
lst_stopwords = nltk.corpus.stopwords.words("english")
def utils_preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=None):
    ## clean (convert to lowercase and remove punctuations and  characters and then strip)
    url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    text = re.sub(url_pattern, '', text)
    ## Tokenize (convert from string to list)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())

    lst_text = text.split()    
    ## remove Stopwords
#     print(len(lst_text))
    if lst_stopwords is not None:
        if (len(lst_text)>10):
            lst_text = [word for word in lst_text if word not in lst_stopwords]
            
    ## back to string from list
    text = " ".join(lst_text)
    symbols_to_remove = "!@#$%^&*カリフォルニアサンディエゴs"
    pattern = "[" + re.escape("".join(symbols_to_remove)) + "]"
    
    # Use re.sub to replace the matched symbols with an empty string
    text = re.sub(pattern, "", text)
    return text

In [22]:
output['factText'] = output['factText'].apply(lambda x:
                                    utils_preprocess_text(x, flg_stemm=False, flg_lemm=True, lst_stopwords=lst_stopwords))

In [36]:
output["importance_count"] = output["importance_count"] / output["importance_count"].max()
output["importance_sum"] = output["importance_sum"] / output["importance_sum"].max()

In [37]:
output = output[output["importance_sum"] >=0]

In [38]:
output

,requestID,factText,unixTimestamp,importance_count,importance_sum,sources,streamID,informationNeeds
0,CrisisFACTS-001-r3,Good to be home again! @ San Diego Internation...,1512632372,0.083333,0.122676,[CrisisFACTS-001-Twitter-48717-0],CrisisFACTS-001-Twitter-48717-0,"[CrisisFACTS-General-q001, CrisisFACTS-Wildfir..."
1,CrisisFACTS-001-r3,#LilacFire Smokey here in Oceanside above the ...,1512681837,0.083333,0.126459,[CrisisFACTS-001-Twitter-5831-0],CrisisFACTS-001-Twitter-5831-0,"[CrisisFACTS-General-q001, CrisisFACTS-Wildfir..."
2,CrisisFACTS-001-r3,#lilacFire photos from Palomar airport rd Carl...,1512679976,0.041667,0.073228,[CrisisFACTS-001-Twitter-512-0],CrisisFACTS-001-Twitter-512-0,[CrisisFACTS-General-q001]
3,CrisisFACTS-001-r3,Back to Cali🌈 . . . #california #sandiego #カリフ...,1512609067,0.041667,0.070275,[CrisisFACTS-001-Twitter-22676-0],CrisisFACTS-001-Twitter-22676-0,[CrisisFACTS-General-q001]
4,CrisisFACTS-001-r3,I'm at San Diego International Airport in San ...,1512605323,0.041667,0.067550,[CrisisFACTS-001-Twitter-49822-0],CrisisFACTS-001-Twitter-49822-0,[CrisisFACTS-General-q001]
...,...,...,...,...,...,...,...,...
466774,CrisisFACTS-018-r5,"If you think differently, go propose a new tax...",1583540053,0.041667,0.032877,[CrisisFACTS-018-Reddit-7515-2],CrisisFACTS-018-Reddit-7515-2,[CrisisFACTS-General-q040]
466775,CrisisFACTS-018-r5,You could get better at running with conscious...,1583585050,0.041667,0.019757,[CrisisFACTS-018-Reddit-7533-2],CrisisFACTS-018-Reddit-7533-2,[CrisisFACTS-General-q041]
466776,CrisisFACTS-018-r5,"They turned me away today, said they had too m...",1583544469,0.041667,0.046803,[CrisisFACTS-018-Reddit-7521-0],CrisisFACTS-018-Reddit-7521-0,[CrisisFACTS-General-q044]
466777,CrisisFACTS-018-r5,Tornados are over and done quickly.,1583562502,0.166667,0.195366,[CrisisFACTS-018-Reddit-7528-6],CrisisFACTS-018-Reddit-7528-6,"[CrisisFACTS-Tornado-q002, CrisisFACTS-Tornado..."


In [39]:
with open("./submission_v1_pre_BM25_Emb_WP.json", "w") as out_file:
    for idx,row in output\
        .drop(columns=["importance_sum"])\
        .rename(columns={"importance_count": "importance"})\
        .iterrows():
        
        out_file.write("%s\n" % json.dumps(dict(row)))

with open("./submission_v2_pre_BM25_Emb_WP.json", "w") as out_file:
    for idx,row in output\
        .drop(columns=["importance_count"])\
        .rename(columns={"importance_sum": "importance"})\
        .iterrows():
        
        out_file.write("%s\n" % json.dumps(dict(row)))

In [40]:
with open("./submission_v2_pre_BM25_Emb_WP.json", "r") as in_file:
    submission_rows = [json.loads(line) for line in in_file]
submission_df = pd.DataFrame(submission_rows)

In [41]:
submission_df

,requestID,factText,unixTimestamp,importance,sources,streamID,informationNeeds
0,CrisisFACTS-001-r3,Good to be home again! @ San Diego Internation...,1512632372,0.122676,[CrisisFACTS-001-Twitter-48717-0],CrisisFACTS-001-Twitter-48717-0,"[CrisisFACTS-General-q001, CrisisFACTS-Wildfir..."
1,CrisisFACTS-001-r3,#LilacFire Smokey here in Oceanside above the ...,1512681837,0.126459,[CrisisFACTS-001-Twitter-5831-0],CrisisFACTS-001-Twitter-5831-0,"[CrisisFACTS-General-q001, CrisisFACTS-Wildfir..."
2,CrisisFACTS-001-r3,#lilacFire photos from Palomar airport rd Carl...,1512679976,0.073228,[CrisisFACTS-001-Twitter-512-0],CrisisFACTS-001-Twitter-512-0,[CrisisFACTS-General-q001]
3,CrisisFACTS-001-r3,Back to Cali🌈 . . . #california #sandiego #カリフ...,1512609067,0.070275,[CrisisFACTS-001-Twitter-22676-0],CrisisFACTS-001-Twitter-22676-0,[CrisisFACTS-General-q001]
4,CrisisFACTS-001-r3,I'm at San Diego International Airport in San ...,1512605323,0.067550,[CrisisFACTS-001-Twitter-49822-0],CrisisFACTS-001-Twitter-49822-0,[CrisisFACTS-General-q001]
...,...,...,...,...,...,...,...
466774,CrisisFACTS-018-r5,"If you think differently, go propose a new tax...",1583540053,0.032877,[CrisisFACTS-018-Reddit-7515-2],CrisisFACTS-018-Reddit-7515-2,[CrisisFACTS-General-q040]
466775,CrisisFACTS-018-r5,You could get better at running with conscious...,1583585050,0.019757,[CrisisFACTS-018-Reddit-7533-2],CrisisFACTS-018-Reddit-7533-2,[CrisisFACTS-General-q041]
466776,CrisisFACTS-018-r5,"They turned me away today, said they had too m...",1583544469,0.046803,[CrisisFACTS-018-Reddit-7521-0],CrisisFACTS-018-Reddit-7521-0,[CrisisFACTS-General-q044]
466777,CrisisFACTS-018-r5,Tornados are over and done quickly.,1583562502,0.195366,[CrisisFACTS-018-Reddit-7528-6],CrisisFACTS-018-Reddit-7528-6,"[CrisisFACTS-Tornado-q002, CrisisFACTS-Tornado..."


In [42]:
submission_df["importance"].max()

1.0

In [43]:
submission_df["importance"].min()

0.0016935331124195224

In [44]:
top_k = 100

for reqId, group in submission_df.groupby("requestID"):
    print(reqId, group.shape[0])
    
    summary = " ".join(group.sort_values(by="importance", ascending=False).head(top_k)["factText"])
    
    print(summary)
    
    break

CrisisFACTS-001-r10 849
I'd love to hear what others have thought of to help save their home, and to expidite an emergency evacuation. People who never played at the level of people they are critiquing need to SIT DOWN. There is a reason why they are… https://t.co/Yw9JQUjxUd WE DO NOT NEED any more donations for animals displaced by #LilacFire or volunteers at the stable area of the… https://t.co/6Vl4RqU3N1 Firefighter from San Diego area killed in Thomas fire When you have an emergency need for a computer and ur PC won't… https://t.co/1YUVkRPdBt We sent around 40 food baskets to Christian Community Services Agency. Glad to help our neighbors in need. #advent… https://t.co/1pDomH5S0Y #LilacFire is getting close to full containment thanks to the hard work of firefighters. CAL FIRE update:… https://t.co/bpS63yOZ8g They did get way too close for comfort; we had fires and homes lost within a mile of us in two directions. Funny how the people I used to hate are now close friends and the one